In [4]:
from collections import namedtuple
import os 

In [2]:
DataIngestionConfig = namedtuple("DataIngestionConfig", [
    "root_dir",
    "source_URL",
    "local_data_file",
    "unzip_dir"
])

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [9]:
os.chdir("../")

In [10]:
%pwd

'd:\\Projects\\Learning\\dogCatClassifier'

In [11]:
from dogCatClassifier.constants import *
from dogCatClassifier.utils import read_yaml, create_directories

In [12]:
class ConfigurationManager:
    def __init__(self,
                 config_file_path = CONFIG_FILE_PATH,
                 params_file_path = PARAMS_FILE_PATH):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )
        return data_ingestion_config

In [13]:
import os
import urllib.request as request
from zipfile import ZipFile

In [14]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )

    def _get_updated_list_of_files(self, list_of_files):
        return [f for f in list_of_files if f.endswith(".jpg") and ("Cat" in f or "Dog" in f)]

    def unzip_and_clean_file(self):
        with ZipFile(file=self.config.local_data_file, mode="r") as zf:
            list_of_files = zf.namelist()
            # Get only images in Dog and Cat folder
            updated_list_of_files = self._get_updated_list_of_files(list_of_files)
            
            for file_path in updated_list_of_files:
                zf.extract(file_path, path=self.config.unzip_dir)
                new_file_path = Path(os.path.join(self.config.unzip_dir, file_path))

                if os.path.getsize(new_file_path) == 0:
                    print(f"Removing 0KB file in {new_file_path}")
                    os.remove(new_file_path)            

In [15]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config= data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.unzip_and_clean_file()
except Exception as e:
    raise e

[2023-12-10 18:51:34,271: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-12-10 18:51:34,288: INFO: common: yaml file: params.yaml loaded successfully]
[2023-12-10 18:51:34,288: INFO: common: created directory at: artifacts]
[2023-12-10 18:51:34,296: INFO: common: created directory at: artifacts/data_ingestion]
